In [124]:
import numpy as np
from sympy import *

from qiskit import *
from qiskit.quantum_info import *

from qiskit.circuit.library import *

from numpy.testing import assert_almost_equal as aae

In [125]:
mat_close = lambda A, B: np.all(np.isclose(A, B))

The $\gamma$ map is defined as $\gamma: u \mapsto u\sigma_y^{\otimes 2} u^T \sigma_y^{\otimes 2}$. Given a matrix $u \in SU(4)$, I found some pattern in the relationship between the trace of $\gamma(u)$ and the number of CNOT it uses. More specifically, let $\alpha = Tr(\gamma(u))$.

- $|\alpha| = 4 \implies$ 0 CNOT

- $|\alpha| = 0 \implies$ 1 CNOT

- $|\alpha| \text{ is real} \implies$ 2 CNOT

- $|\alpha| \text{ is complex} \implies$ 3 CNOT

Whether this implication is always true is in question. It's kinda late so I will probably prove this tomorrow.

In [294]:
def to_su(u): # Convert a matrix in U(4) to SU(4)
    u = u / np.sqrt(np.sqrt(np.linalg.det(u)))
    
    return u

def gamma_map(u): # Perform the gamma map above
    sigma_y = np.array([[0, -1j], 
                        [1j, 0]])
    
    return u@np.kron(sigma_y, sigma_y)@u.T@np.kron(sigma_y, sigma_y)

def classification(u, output_trace = False):
    
    u = to_su(u)
    
    mat_trace = np.round(np.trace(gamma_map(u)), 14)
    
    if output_trace:
        print(mat_trace)
    
    if (mat_trace == -4) or (mat_trace == 4):
        return 0
    elif mat_trace == 0:
        return 1
    elif np.isreal(mat_trace):
        return 2
    else:
        return 3

I check the classification for CNOT as two qubit gate.

In [286]:
cnot = np.array([[1, 0, 0, 0], 
                 [0, 1, 0, 0], 
                 [0, 0, 0, 1], 
                 [0, 0, 1, 0]])

for i in range(1000):

    MA = np.kron(random_unitary(2).data, random_unitary(2).data)
    MB = np.kron(random_unitary(2).data, random_unitary(2).data)
    MC = np.kron(random_unitary(2).data, random_unitary(2).data)
    MD = np.kron(random_unitary(2).data, random_unitary(2).data)

    M0 = MA
    M1 = M0@cnot@MB
    M2 = M1@cnot@MC
    M3 = M2@cnot@MD

    assert classification(M0) == 0, 'zero'
    assert classification(M1) == 1, 'one'
    assert classification(M2) == 2, 'two'
    assert classification(M3) == 3, 'three'

I did a little bit more digging and I found that it also works for $iSWAP$ gate.

In [300]:
isw = iSwapGate().to_matrix()

for i in range(1000):

    MA = np.kron(random_unitary(2).data, random_unitary(2).data)
    MB = np.kron(random_unitary(2).data, random_unitary(2).data)
    MC = np.kron(random_unitary(2).data, random_unitary(2).data)
    MD = np.kron(random_unitary(2).data, random_unitary(2).data)

    M0 = MA
    M1 = M0@isw@MB
    M2 = M1@isw@MC
    M3 = M2@isw@MD

    assert classification(M0) == 0, 'zero'
    assert classification(M1) == 1, 'one'
    assert classification(M2) == 2, 'two'
    assert classification(M3) == 3, 'three'

I look at the $\sqrt{iSWAP}$ as the native two qubit gates. Here, the classification changes a little. For 1 gate, the trace is 2. For 2 and 3, the trace is complex. This might have to do with the fact that two $\sqrt{iSWAP}$ gate makes up a 79% of the SU(4) gate.

In [320]:
sqisw = np.array([[1, 0, 0, 0],
                  [0, 1/np.sqrt(2), 1j/np.sqrt(2), 0], 
                  [0, 1j/np.sqrt(2), 1/np.sqrt(2), 0], 
                  [0, 0, 0, 1]])

MA = np.kron(random_unitary(2).data, random_unitary(2).data)
MB = np.kron(random_unitary(2).data, random_unitary(2).data)
MC = np.kron(random_unitary(2).data, random_unitary(2).data)
MD = np.kron(random_unitary(2).data, random_unitary(2).data)

M0 = MA
M1 = M0@sqisw@MB
M2 = M1@sqisw@MC
M3 = M2@sqisw@MD

assert classification(M0) == 0, 'zero'
assert classification(M1) == 1, 'one'
assert classification(M2) == 2, 'two'
assert classification(M3) == 3, 'three'

AssertionError: one